In [1]:
import os
import sys

sys.path.insert(0, "\\".join(os.path.abspath(os.curdir).split("\\")[:-1:]))
from pathlib import Path

from copy import deepcopy

from src.utils.utils import *

# Pré processar os metadados

In [2]:
from copy import deepcopy

def rename_keys(dictionary):
    dict_keys_to_rename = {
        "id1": "patient_id",
        "leftright": "left_or_right_breast",
        "abnormality": "abnormality_type",
        "classification": "pathology",
        "reference_number": "patient_id",
        "laterality": "left_or_right breast",
        "view": "image_view",
        "assessment": "bi-rads",
        "age": "patient_age",
        "acr": "breast_density"
    }

    keys_order = dictionary.keys()
    
    for key in dictionary.keys():
        key_lower = deepcopy(key)
        key_lower = key_lower.lower().replace(" ", "_")
        
        if key in dict_keys_to_rename.keys():
            keys_order = list(map(lambda x: x.replace(key, dict_keys_to_rename[key_lower]), keys_order))
            
    new_dict = {}
    for key_order, key in zip(keys_order, dictionary.keys()):
        new_dict[key_order.lower().replace(" ", "_").replace("-", "_")] = dictionary[key]
           
    return new_dict

### DDSM

In [3]:
studies = []
json_names = ['calc_case_description_test_set', 'calc_case_description_train_set', 
              'mass_case_description_test_set', 'mass_case_description_train_set']
list_id = set()

list_metadata = []
path_metadata = "../metadata/mamografia/extracted_metadata/csv_and_dicom_DSSM"

# Pega e junta todos os metadados
for json_name in json_names:
    list_metadata = list_metadata + load_json(json_name, path_metadata)
    
# Pega os id's de todos os pacientes    
for metadata in list_metadata:
    list_id.add(metadata["metadata_csv"]["patient_id"])
    
# Itera sobre todos os pacientes    
for id in list_id:
    list_patient_metadata = []
    new_metadata = {}
    
    # Pega todos os metadados associados a um paciente
    for metadata in list_metadata:
        if id == metadata["metadata_csv"]["patient_id"]:
            list_patient_metadata.append(metadata)
    
    # Cria o objeto paciente  
    tmp = list_patient_metadata[0]["metadata_dicom_files"]['original'][0]   
    patient = {'sex': tmp["patient's_sex (0010, 0040)"],
               'birth_date': tmp["patient's_birth_date (0010, 0030)"]}
    
    new_metadata['researcher'] = {"name": "", "email": "", "orcid": "", "lattes": ""}
    new_metadata['patient'] = patient
    images = []
    
    # Cria a lista de imagens
    for patient_metadata in list_patient_metadata:
        metadata_csv = patient_metadata['metadata_csv']
        
        image = {'orientation': metadata_csv['image view']}
        
        # Pega o path da imagem original
        directory = Path(metadata_csv["original_image_path"])
        path_original_image = list(directory.rglob("*.dcm*"))[0]
        
        # Salva o path e classificação na imagem
        image['original_image_path'] = metadata_csv['original_image_path']
        image['classification'] = metadata_csv['pathology']
        
        # Processamento para o objeto image_metadata
        tmp = deepcopy(metadata_csv)
        del tmp['original_image_path']
        del tmp['cropped_image_path']
        del tmp['image view']
        del tmp['patient_id']
        del tmp['pathology']
        
        # Cria o objeto image_metadata
        image_metadata = {'study_name': 'DDSM'} | tmp
        image['image_metadata'] = image_metadata
        
        # Pega os paths das imagens cropped
        directory = Path(metadata_csv["cropped_image_path"])
        paths_processed_images = list(directory.rglob("*.dcm*"))
        
        # Cria o objeto image_processing
        processings = [{'processing_type': 'Cropped', 'result_image_type': str(path)} for path in paths_processed_images]
        image['image_processing'] = processings
        
        # Cria o objeto image_type
        image['image_type'] = {'label': 'DICOM', 'description': '(tag_group, tag_element)',
                      'data': patient_metadata['metadata_dicom_files']['original']}
        
        images.append(image)
        
    new_metadata['image'] = images
    
    studies.append(new_metadata)

save_json("metadata", studies, "./mongodb_database") 

### CMMD

In [5]:
"""#CMMD e INBREAST     
database_names = ['CMMD', 'INBREAST']
json_names = ['csv_and_dicom_CMMD', 'csv_and_dicom_INBREAST']

for database_name, json_name in zip(database_names, json_names):
    list_metadata = load_json(json_name, "../metadata/mamografia/extracted_metadata")
    
    for metadata in list_metadata: 
        new_metadata = {}
        new_metadata['database_name'] = database_name
        new_metadata['study_name'] = metadata['study_name']
        
        metadata_csv = rename_keys(metadata['metadata_csv'])
    
        new_metadata = new_metadata | metadata_csv
        new_metadata['metadata_dicom_files'] = metadata['metadata_dicom_files']
        
        studies.append(new_metadata)
    
save_json("metadata", studies, "./mongodb_database") """

' #CMMD e INBREAST     \ndatabase_names = [\'CMMD\', \'INBREAST\']\njson_names = [\'csv_and_dicom_CMMD\', \'csv_and_dicom_INBREAST\']\n\nfor database_name, json_name in zip(database_names, json_names):\n    list_metadata = load_json(json_name, "../metadata/mamografia/extracted_metadata")\n    \n    for metadata in list_metadata: \n        new_metadata = {}\n        new_metadata[\'database_name\'] = database_name\n        new_metadata[\'study_name\'] = metadata[\'study_name\']\n        \n        metadata_csv = rename_keys(metadata[\'metadata_csv\'])\n    \n        new_metadata = new_metadata | metadata_csv\n        new_metadata[\'metadata_dicom_files\'] = metadata[\'metadata_dicom_files\']\n        \n        studies.append(new_metadata)\n    \nsave_json("metadata", studies, "./mongodb_database") '